#SAI SRIKANTH SARABU
#A20343781 

# Project-2: Locality Sensitive Hashing

In [56]:
import random
import os
import numpy as np

from random import randrange
from shutil import copyfile

In [57]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Suspicious datasets for this project will be in <b>Suspicious</b> directory


Your query datasets will be in <b>Original</b> directory


You have to use any one original Wikipedia article from <b>Original</b> for <b>Fact Checks</b> steps

In [58]:
data = '/content/drive/MyDrive/corpus_data/' 

#creating 2 directories with Suspicious and Original as their name and placing the files respectively
try:
  os.mkdir('Suspicious')
  os.mkdir('Original')
except:
  print('folders already created')

for f in os.listdir(data):
  if 'orig' in f:
    copyfile(data + f, 'Original/'+f)
  else:
    copyfile(data + f, 'Suspicious/'+f)

folders already created


### STEP - 1: Shingling (20 points)

In [59]:
# Type your code here... 
# Create necessary number of cells below this cell
k = 3

# NOTE: No complex text processing is required
# convert just upper case characters to lower case

original_path = '/content/Original'
suspicious_path = '/content/Suspicious'

uniquek3shingles = set()                    #Created 3 empty sets for 3, 4 and 5 unique shingles
uniquek4shingles = set()
uniquek5shingles = set()
paths = [suspicious_path, original_path]    

#Below for loop with traverse through all the documents both Original and Suspicious and create 3, 4 and 5 shingles.
for path in paths:                          
  for file in os.listdir(path):
    k=3
    with open(os.path.join(path, file), "r",encoding ='utf8',errors='ignore') as filename:     #reading each file from folder
      pList = filename.read()
      wList = pList.split()                             #splits the file into words and stored into the list
    for i in range(len(wList)-k+1):                     #looping through all the words
      w = wList[i:i + k]                                #creating the 3 words list 
      x = ' '.join(w).lower()                           #concatenating them with space separated 
      uniquek3shingles.add(x)                           #adding to the set
    k=4                                                 #increment k
    for i in range(len(wList)-k+1):                     #below 2 loops also perform same with k incremented
      w = wList[i:i + k]
      x = ' '.join(w).lower()
      uniquek4shingles.add(x)
    k=5
    for i in range(len(wList)-k+1):
      w = wList[i:i + k]
      x = ' '.join(w).lower()
      uniquek5shingles.add(x)

In [60]:
print('Number of unique 3-shingles are:',len(uniquek3shingles))
print('Number of unique 4-shingles are:',len(uniquek4shingles))
print('Number of unique 5-shingles are:',len(uniquek5shingles))

Number of unique 3-shingles are: 13724
Number of unique 4-shingles are: 15127
Number of unique 5-shingles are: 15796


In [76]:
# Type your code to get the 5-shingle index here
shingleIndex5 = {}      #created 2 dictionaries one stores 5 shingles of each file from suspicious folder, other 
OrignalIndex5 = {}      #stores 5 shingles from each orginal file 
for path in paths:
  for file in os.listdir(path):                        
    with open(os.path.join(path, file), "r",encoding ='utf8',errors='ignore') as filename:
      pList = filename.read().lower()       
      wList = pList.split()
      k=5
    k5shinglesIndex = set()                     #created temporary set to store shingles from each file
    for i in range(len(wList)-k+1):
      w = wList[i:i + k]
      x = ' '.join(w).lower()
      k5shinglesIndex.add(x)
    if 'orig' in file:                          #checking the file name and placing the shingles in respective dictionary
      OrignalIndex5[file] = list(k5shinglesIndex)   #making shingle into list and adding to a dictionary with key as file name
    else:
      shingleIndex5[file] = list(k5shinglesIndex)

uniquek5shinglesList = list(uniquek5shingles)   #converting set of all files 5 shingles to set.
uniquek5shinglesList

['splitting the programming into optimal',
 'try to fool the search',
 'further vector space models, basically',
 'for that page. the pagerank',
 'also be refered to as',
 'time he accidently records the',
 'of the train and car',
 'categorization is a powerful mechanism',
 'space model include: generalized vector',
 'failure of the vector space',
 'the definition of term depends',
 'classes and to go into',
 'is not sufficiently mature, may',
 'modify since they do not',
 'simpler term count model the',
 'a zero value for the',
 'for many programmerers when developing',
 'observations, for instance the probability',
 'vectors will not have any',
 'existing code without modification. inheritance',
 'is still a good indicator',
 'a number of other models',
 'exposed by ancestor, or by',
 'not have much in common.',
 'original form of pagerank initial',
 'classes). categorization in computer languages',
 'and had no match which',
 'probability requires that (1) where',
 'the pagerank dep

### STEP - 2: Min-Hashing (40 points)

In [62]:
# ***************************************************NEW***********************************************************
# Generate Hash functions - 
    # We use (ax + b) mod N formula to permute shingle index
    # where a,b are random numbers, N total index size, and x is the index
# We need to do L permutations - In other words we need to have L permutations (lists) of new indexes
# Following function takes total index size N and L as arguments
    # And returns L new lists of size N
    
def get_hash_functions(N,L):
    hash_functions = []
    
    for itr in range(L):
        a=randrange(1,400)
        b=randrange(1,400)
        
        new_hash_function = []
        for i in range(N):
            new_hash_function.append((a * i + b) % N)
        
        hash_functions.append(new_hash_function)
    return hash_functions
        
# test
# hash_functions = get_hash_functions(5000,50)

In [63]:
# Type your code here to generate all L hash functions
# Generate hash functions only for shingle index created for k=5
#L = 50
L = [50,100,200,500,1000]                         #created a list of L values given in quetion
N = len(uniquek5shingles)                         #taking number of unique shingles as length of hashfunction
allHFunList = []
for i in L:
  allHFunList.append(get_hash_functions(N,i))     #calling hash function each L

In [64]:
# Type your code here to get the final signature matrix S here

#below function will generate signature matric for each L ranging from 50, 100, 200, 500 and 1000
def getSignatureVectors(shingleIndex, L, hashfuncs):   
  signdict = {}                                   #created dictionary to store file name as key and signature vector as value.
  for key in shingleIndex:                        #for each file in shingle index
    s1=[np.inf for n in range(L)]                 #Initializing each value in signature vector as infinity
    for s in uniquek5shingles:                    #for each shingle in all unque shingles
      if s in shingleIndex[key]:                  #for each shingle in the file
        index=uniquek5shinglesList.index(s)       #getting the index from unique shingle list 
        for i in range(L):                        #for each of the hash function
          if hashfuncs[i][index]<s1[i]:           #checking the hash value with signature vector
            s1[i]=hashfuncs[i][index]             #assigning hashvalue if it less than already exsisted value
    signdict[key] = s1                            #assigning the signature vector to the dictionary
  return signdict

Suspicioussignatures = []                         #created an empty list to store all suspicious signature vectors for all L's 
for i in range(len(allHFunList)):
  signMat = getSignatureVectors(shingleIndex5, L[i], allHFunList[i])
  Suspicioussignatures.append(signMat)            #appending to the list
#Suspicioussignatures[0]

In [65]:
# Type your code here to do the fact check 
#      with any one query document in the 'Original' directory

original = 'Original/orig_taska.txt'    #took file a in as it is mentioned any file in quetion, you can change it any file in original directory

# STEP-1: Generate 5-shingles 
    # (if any shingles are not present in your shingle index, simply ignore them)

#below lines will generate 5 shingle index for the file given above
k5ShigleIndextaskdict = {}
k5ShigleIndextask = set()
with open(original, "r",encoding ='utf8',errors='ignore') as filename:
  pList = filename.read()
  wList = pList.split()
k = 5
for i in range(len(wList)-k+1):
  w = wList[i:i + k]
  x = ' '.join(w).lower()
  k5ShigleIndextask.add(x)
#print(k5ShigleIndextask)
k5ShigleIndextaskdict[original.split('/')[1]] = list(k5ShigleIndextask)
    
    
# STEP-2: Generate signature vector from L hash functions

taskcSignMatrix = []                    #generating signatue vector for the above given original file for all L values
for i in range(len(allHFunList)):
  signMat = getSignatureVectors(k5ShigleIndextaskdict, L[i], allHFunList[i])
  taskcSignMatrix.append(signMat)


# STEP-3: Calculate Jaccard similarity of signature vector of orginal doc.
    # and all other documents 
    
t = 0.85              #change the threshold to get more or less pairs of files.

dictTaskJSAllLs = {}                                 #created an empty dictionaly to store all the jaccard similarity greater than given t
for l in range(len(L)):                              #looping through all L value range
  dictTaskJSeachL = {}                               #dictionary to store all file pairs as key and jaccard similarity as value
  for i in Suspicioussignatures[l]:                  #looping through each suspicious vectors from each L 
    for j in taskcSignMatrix[l]:                     #looping through original file vectors from each L
      count =0                                       #initializing temporary count variable to check jaccard similarity 
      for r in range(L[l]):                          #looping through each value in signature vector
        if Suspicioussignatures[l][i][r] == taskcSignMatrix[l][j][r]:   #comparing values from suspicious signature vector to original signature vector 
          count+=1                                   #incrementing the count if same
      js = count/L[l]                                #dividing the count with Hash functions to get jaccard similarity
      if js> t:                                      #checking whether Jaccard similarity is greater than threshold
        dictTaskJSeachL[str(i)+' ,'+str(j)] = js     #creating a dictioanary with key as pair of file and value as its Jaccard similarity
  dictTaskJSAllLs[L[l]] = dictTaskJSeachL            #dictionary to store key as L value and value as all the pairs with JS greater than JS

# STEP-4: Facts Checks
# For each L = {50,100,200,500,1000}, report all documents (file_names) below that have Jaccard similarity > t=0.85
# Sort the documents in decreasing order of the Jaccard similarity

#below loop will print Jaccard similarity in decreasing order for each L value
for i in dictTaskJSAllLs: 
  print('\033[1m'+'Jaccard Similarity for L = '+str(i)+' in decreasing order for t = '+str(t)+'\033[0m')
  if len(dictTaskJSAllLs[i])>0:
    orderedValuesJS = dict(sorted(dictTaskJSAllLs[i].items(), key=lambda itm: itm[1], reverse = True))
    for j in orderedValuesJS:
      print(j+'-> '+'\033[1m'+str(orderedValuesJS[j])+'\033[0m')
  else:
    print('No document found with jaccard similarity greater than ',t)
  print('\n')

Jaccard Similarity for L = 50 in decreasing order for t = 0.85
g4pC_taska.txt ,orig_taska.txt-> 0.92
g0pE_taska.txt ,orig_taska.txt-> 0.86


Jaccard Similarity for L = 100 in decreasing order for t = 0.85
No document found with jaccard similarity greater than  0.85


Jaccard Similarity for L = 200 in decreasing order for t = 0.85
g0pE_taska.txt ,orig_taska.txt-> 0.89
g4pC_taska.txt ,orig_taska.txt-> 0.86


Jaccard Similarity for L = 500 in decreasing order for t = 0.85
g0pE_taska.txt ,orig_taska.txt-> 0.874


Jaccard Similarity for L = 1000 in decreasing order for t = 0.85
g0pE_taska.txt ,orig_taska.txt-> 0.88




In [66]:
originalSignatures = []                           #created an empty list to store all original signature vectors for all L's 
for i in range(len(allHFunList)):
  signMat = getSignatureVectors(OrignalIndex5, L[i], allHFunList[i])
  originalSignatures.append(signMat)

In [67]:
#here we are taking all the original files and finding jaccard similarity
dictJSAllLs = {} 
for l in range(len(L)):
  dictJSeachL = {}
  for i in Suspicioussignatures[l]: 
    for j in originalSignatures[l]:
      count =0
      for r in range(L[l]):
        if Suspicioussignatures[l][i][r] == originalSignatures[l][j][r]:
          count+=1
      js = count/L[l]
      if js> t:
        dictJSeachL[str(i)+' ,'+str(j)] = js
  dictJSAllLs[L[l]] = dictJSeachL

for i in dictJSAllLs: 
  print('\033[1m'+'Jaccard Similarity for L = '+str(i)+' in decreasing order for t = '+str(t)+'\033[0m')
  if len(dictJSAllLs[i])>0:
    orderedValuesJS = dict(sorted(dictJSAllLs[i].items(), key=lambda itm: itm[1], reverse = True))
    for j in orderedValuesJS:
      print(j+'-> '+'\033[1m'+str(orderedValuesJS[j])+'\033[0m')
  else:
    print('No document found with jaccard similarity greater than ',t)
  print('\n')

Jaccard Similarity for L = 50 in decreasing order for t = 0.85
g4pC_taska.txt ,orig_taska.txt-> 0.92
g0pE_taska.txt ,orig_taska.txt-> 0.86


Jaccard Similarity for L = 100 in decreasing order for t = 0.85
No document found with jaccard similarity greater than  0.85


Jaccard Similarity for L = 200 in decreasing order for t = 0.85
g0pE_taska.txt ,orig_taska.txt-> 0.89
g4pC_taska.txt ,orig_taska.txt-> 0.86


Jaccard Similarity for L = 500 in decreasing order for t = 0.85
g0pE_taska.txt ,orig_taska.txt-> 0.874


Jaccard Similarity for L = 1000 in decreasing order for t = 0.85
g0pE_taska.txt ,orig_taska.txt-> 0.88




### STEP - 3: LSH (30 points)

In [68]:
# Type your code here to hash signature matrix into B buckets
# Use the technique to split the signature matrix into b bands of r rows
# Convert only the signature matrix generated with L=1000

b = 50
r = 20
B = 199
a = random.sample(range(50), r)     #with generate random 20 numbers in 50 

In [69]:
# Type your code here to do generate candidate documents
# Follow all steps from STEP - 2 fact check (except the Jaccard similarity part)

#below function will hash each band b of r rows of a file and places into one bucket of 199 buckets 
def getBuckets(matrixSign, b, B):
  buckets = []                                 #initializing empty list to store buckets of dictionries for each band b
  for i in range(b):                           #looping through all bands
    bucdict = {}                              #initializing empty dictionary to store bucket number as key and files as list in value
    for j in matrixSign:                      #looping through signature vector for each file 
      hashValue = (np.sum((a)*(np.array(matrixSign[j][i*r:(i*r)+r])))) % B    #here it multiplies each row from band to random number and percentile with B(199) to get bucket number
      if hashValue in list(bucdict.keys()):    #placing the file in the bucket using dictionary and appending to the list
        bucdict[hashValue].append(j) 
      else:
        bucdict[hashValue] = list([j])
    buckets.append(bucdict)
  return buckets
#print(Suspicioussignatures[4])
#here are using already created suspicious files signature and for L=1000
suspiciousBuckets = getBuckets(Suspicioussignatures[4],b,B) #this will hash files from signature vectors to any of the 199 buckets
#print(suspiciousBuckets)

In [70]:
# STEP - 1: Split each original document signature vector into b bands of r rows

#this step is covered in by calling getBuckets() in below function 

# STEP - 2: Hash using the same hash functions created for 
    # signature matrix hashing (in the previous cell)

#calling getbuckets will hash each orinal to any of the 199 buckets
originalBuckets = getBuckets(originalSignatures[4],b,B)   #we are using originalSignatures 
#originalBuckets


In [71]:
# Type your code here to do the fact check
# Calculate Jaccard similarity of the oiginal document with only 
    # candidate documents

#below function will check if any suspicious bucket hash is same as original bucket hash    
candidatesdict = {}
for i in range(b):                                              #for each band
  candidates = [value for value in suspiciousBuckets[i] if value in originalBuckets[i]]  #checking if same value is present respective band
  #print(candidates)
  for j in candidates:                                #looping through all the candidates, which are found in same bucket
    for o in originalBuckets[i][j]:                   #for each original document 
      if o in list(candidatesdict.keys()):            #creating dictionary to store candidate documents, key as original doc, value as set of candidates
        candidatesdict[o].update(set(suspiciousBuckets[i][j])) 
      else:
        candidatesdict[o] = set(suspiciousBuckets[i][j])
#print(candidatesdict)

#this loop is to create nested dictionary with key as original file and value is a dictionary of (key as all suspicious documents and value is Jaccard Similarity)
L1000dictJS = {}                                   #creating a dictionary to store all the jaccard similarity of L=1000
forFalNegUsage = set()                             #this is used while doing false negatives
for j in originalSignatures[4]:                   #looping the same way as dicussed above to get Jaccard Similarity
  suspDictJS = {}
  for i in Suspicioussignatures[4]:
    count =0
    for r in range(L[4]):
      if Suspicioussignatures[4][i][r] == originalSignatures[4][j][r]:
        count+=1
    js = count/L[4]
    if js > t:
      forFalNegUsage.add(i)                       #adding all documents which has jaccard similarity with any original document greater than threshold t
    suspDictJS[i] = js                            #creating a nested dictionary with key as original file and value is a dictionary of (key as all suspicious documents and value is Jaccard Similarity)
  L1000dictJS[j] = suspDictJS

#L1000dictJS

In [72]:
dictCandJSeachL1000 = {}                         #dictionary to store all candidate documents with Jaccard Similarity > threshold t
dictCandJSeachL1000FalPos = {}                   #dictionary to store all suspicious documents mapped to different original documents     
candDocsSetlessThant = set()                     #set to store candidate documents with Jaccard Similarity < threshold t
candDocsSetmoreThant = set()                     #set to store candidate documents with Jaccard Similarity > threshold t
candDocsSet = set()                              #set to store all candidate documents 
for i in candidatesdict:                          #looping through all the candidate documents with keys as original documents
  for j in list(candidatesdict[i]):               #looping through all the values, those candidates(suspicious documents) found aboove
    if L1000dictJS[i][j] >= t:                    #checking jaccard similarity
      dictCandJSeachL1000[j+', '+i] = L1000dictJS[i][j] #dictionary storing key as pair and value Jaccard similarity>t
      candDocsSetmoreThant.add(j)                  
    else:
      dictCandJSeachL1000FalPos[j+', '+i] = L1000dictJS[i][j]  #dictionary storing key as pair and value Jaccard similarity<t
      candDocsSetlessThant.add(j)
    candDocsSet.add(j)    #all candidate documents

print('\033[1m'+'The number of candidate documents are: '+str(len(candDocsSet))+'\033[0m')
#printing all the candidate documents in decending order of Jaccard Similarity
print('\033[1m'+'Jaccard Similarity for candidate documents in decreasing order for given t>='+str(t)+'\033[0m')
if len(dictCandJSeachL1000)>0:                    
    orderedValuesJS = dict(sorted(dictCandJSeachL1000.items(), key=lambda itm: itm[1], reverse = True))
    for j in orderedValuesJS:
      print(j+'-> '+'\033[1m'+str(orderedValuesJS[j])+'\033[0m')
    print('\n')
else:
  print('No document found with jaccard similarity greater than ',t)


The number of candidate documents are: 68
Jaccard Similarity for candidate documents in decreasing order for given t>=0.85
g0pE_taska.txt, orig_taska.txt-> 0.88




#False Positives

In [73]:
#Printing all documents together combining suspicious documents together which fell in same of different original documents
falsePosdocs = candDocsSet.difference(candDocsSetmoreThant) #subtracting the set of documents with JS less than t and from all candidate documents set 
print('\033[1m'+'The number of false positive documents are: '+str(len(falsePosdocs))+'\033[0m')
print('\033[1m'+'The documents in false positives are: '+'\033[0m')
print(falsePosdocs)

The number of false positive documents are: 67
The documents in false positives are: 
{'g4pB_taskd.txt', 'g3pA_taskc.txt', 'g1pA_taskd.txt', 'g4pE_taskb.txt', 'g3pC_taska.txt', 'g4pE_taskc.txt', 'g3pC_taske.txt', 'g2pE_taske.txt', 'g4pB_taske.txt', 'g0pE_taskc.txt', 'g1pA_taskb.txt', 'g4pD_taska.txt', 'g1pD_taskc.txt', 'g4pD_taskb.txt', 'g0pD_taske.txt', 'g0pE_taskb.txt', 'g0pA_taskd.txt', 'g2pC_taske.txt', 'g2pB_taske.txt', 'g3pB_taska.txt', 'g3pC_taskd.txt', 'g1pB_taska.txt', 'g3pB_taskc.txt', 'g4pD_taske.txt', 'g0pE_taskd.txt', 'g1pA_taskc.txt', 'g2pA_taskd.txt', 'g0pB_taska.txt', 'g2pC_taskc.txt', 'g2pE_taskb.txt', 'g3pC_taskb.txt', 'g2pA_taske.txt', 'g2pB_taskc.txt', 'g2pA_taskb.txt', 'g4pD_taskd.txt', 'g0pB_taskb.txt', 'g0pA_taska.txt', 'g0pD_taskd.txt', 'g3pC_taskc.txt', 'g0pA_taskb.txt', 'g0pA_taskc.txt', 'g0pC_taskc.txt', 'g3pB_taske.txt', 'g4pC_taskd.txt', 'g0pD_taskb.txt', 'g1pD_taskd.txt', 'g2pC_taskb.txt', 'g0pC_taskd.txt', 'g0pC_taske.txt', 'g1pB_taskd.txt', 'g4pC_taska.t

In [74]:
#Here I am printing all the suspicious documents that hashed to same bucket of original separately 
print('\033[1m'+'printing candidate documents with each original file separatly\nFalse positives: Candidate documents that fell in same bucket as original document but has jaccard similarity less than '+str(t)+'\033[0m')
#print(len(dictCandJSeachL1000FalPos))
if len(dictCandJSeachL1000FalPos)>0:
    orderedValuesJS = dict(sorted(dictCandJSeachL1000FalPos.items(), key=lambda itm: itm[1], reverse = True))
    for j in orderedValuesJS:
      print(j+'-> '+'\033[1m'+str(orderedValuesJS[j])+'\033[0m')
    print('\n')
else:
  print('No document found')

printing candidate documents with each original file separatly
False positives: Candidate documents that fell in same bucket as original document but has jaccard similarity less than 0.85
g4pC_taska.txt, orig_taska.txt-> 0.833
g4pC_taskd.txt, orig_taskd.txt-> 0.603
g4pB_taske.txt, orig_taske.txt-> 0.522
g3pC_taska.txt, orig_taska.txt-> 0.326
g0pC_taskd.txt, orig_taskd.txt-> 0.285
g3pB_taske.txt, orig_taske.txt-> 0.189
g4pB_taskd.txt, orig_taskd.txt-> 0.169
g3pA_taskc.txt, orig_taskc.txt-> 0.134
g0pB_taskd.txt, orig_taskd.txt-> 0.116
g0pA_taskd.txt, orig_taskd.txt-> 0.053
g4pB_taske.txt, orig_taskb.txt-> 0.053
g2pB_taske.txt, orig_taskd.txt-> 0.046
g3pA_taske.txt, orig_taske.txt-> 0.043
g1pA_taskc.txt, orig_taskc.txt-> 0.04
g2pA_taskb.txt, orig_taske.txt-> 0.04
g3pC_taske.txt, orig_taske.txt-> 0.039
g4pD_taskb.txt, orig_taske.txt-> 0.039
g2pC_taske.txt, orig_taskb.txt-> 0.038
g2pA_taske.txt, orig_taskd.txt-> 0.037
g3pB_taskc.txt, orig_taska.txt-> 0.037
g0pB_taskb.txt, orig_taskd.txt-> 0

#False Negatives

In [75]:
falNegList = []  #initializing empty list to add false negatives
for i in forFalNegUsage:              #looping though all the suspicious documents with Jaccard Similarity> threshold
  if i not in candDocsSet:            #if not present in candidate documents then adding to the set        
    falNegList.append(i) 
#printing false negatives
if len(falNegList)>0:
  print('\033[1m'+'The number of false negatives documents are: '+str(len(falNegList))+'\033[0m')
  print('\033[1m'+'The documents in false positives are: '+'\033[0m')
  print(falNegList)
else:
  print('No document found')

No document found
